# 1. Project Overview & Business Understanding



The head of a new movie studio requires data-driven recommendations to guide initial film production choices, specifically aiming to maximize worldwide box office success.



Key Questions to be Answered:



1. Which film genres yield the highest average worldwide revenue?


2. How does audience reception (IMDB rating) correlate with financial success?


3. Is there an optimal film runtime that maximizes gross earnings?


4. How consistent are audience ratings within the highest-grossing film genres?


5. Are the observed differences in average genre revenue statistically significant, or merely due to random chance?

2. Data Understanding and Acquisition

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import sqlite3

In [26]:
# Establish connection to the IMDB database
conn = sqlite3.connect('im.db')
# Load the Box Office Mojo (BOM) gross revenue data
df = pd.read_csv('bom.movie_gross.csv')

In [27]:
# Display all tables in the database
query1 = """
SELECT * 
FROM sqlite_master
 WHERE type = 'table';
 """
print(pd.read_sql_query(query1, conn))

     type                 name             tbl_name  rootpage  \
0   table         movie_basics         movie_basics         2   
1   table            directors            directors         3   
2   table            known_for            known_for         4   
3   table           movie_akas           movie_akas         5   
4   table        movie_ratings        movie_ratings         6   
5   table              persons              persons         7   
6   table           principals           principals         8   
7   table              writers              writers         9   
8   table      movies_from_csv      movies_from_csv     41432   
9   table  bom.movie_gross.csv  bom.movie_gross.csv     41369   
10  table              revenue              revenue     41372   

                                                  sql  
0   CREATE TABLE "movie_basics" (\n"movie_id" TEXT...  
1   CREATE TABLE "directors" (\n"movie_id" TEXT,\n...  
2   CREATE TABLE "known_for" (\n"person_id" TEXT,\.

In [28]:
print("\nBox Office Mojo Data Preview:")
# Preview BOM data structure
print(df.head())


Box Office Mojo Data Preview:
                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
3     535700000  2010  
4     513900000  2010  


3. Data Preparation and Cleaning

This section integrates and cleans the data from the IMDB database and the Box Office Mojo CSV file. The key step is joining movie metadata (runtime, genre, rating) with financial gross data.

In [29]:
# This creates a 'Revenue' table inside the im.db


df.to_sql("revenue", conn, if_exists="replace", index=False)

In [30]:
# --- Data Merging: SQL Query to join all necessary tables -
query2= """ 
SELECT mb.movie_id, r.title, mb.original_title,r.year, mb.runtime_minutes,mb.genres, r.studio, r.domestic_gross, r.foreign_gross, mr.averagerating, mr.numvotes
FROM movie_basics AS mb
JOIN Revenue AS r
ON mb.primary_title = r.title
JOIN movie_ratings AS mr
ON mb.movie_id =  mr.movie_id;
"""
df1=pd.read_sql_query(query2, conn)
df1.head()

,movie_id,title,original_title,year,runtime_minutes,genres,studio,domestic_gross,foreign_gross,averagerating,numvotes
0,tt0315642,Wazir,Wazir,2016,103.0,"Action,Crime,Drama",Relbig.,1100000.0,None,7.1,15378
1,tt0337692,On the Road,On the Road,2012,124.0,"Adventure,Drama,Romance",IFC,744000.0,8000000,6.1,37886
2,tt0359950,The Secret Life of Walter Mitty,The Secret Life of Walter Mitty,2013,114.0,"Adventure,Comedy,Drama",Fox,58200000.0,129900000,7.3,275300
3,tt0365907,A Walk Among the Tombstones,A Walk Among the Tombstones,2014,114.0,"Action,Crime,Drama",Uni.,26300000.0,26900000,6.5,105116
4,tt0369610,Jurassic World,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",Uni.,652300000.0,"1,019.4",7.0,539338


In [31]:
# Data Merging: SQL Query to join all necessary tables.
query_all = """
SELECT
t1.movie_id,
t1.primary_title,
t1.start_year,
t1.runtime_minutes,
t1.genres,
t2.averagerating,
t2.numvotes,
t3.domestic_gross,
REPLACE(t3.foreign_gross, ',', '') AS foreign_gross,
(t3.domestic_gross + CAST(REPLACE(t3.foreign_gross, ',', '') AS REAL)) AS Total_revenues
FROM movie_basics t1
JOIN movie_ratings t2
ON t1.movie_id = t2.movie_id
JOIN Revenue t3
ON t1.primary_title = t3.title;
"""
# Execute query and create the final working DataFrame
df1 = pd.read_sql_query(query_all, conn)
df1.head() # checks the first 5 rows of the merged dataframe

,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes,domestic_gross,foreign_gross,Total_revenues
0,tt0315642,Wazir,2016,103.0,"Action,Crime,Drama",7.1,15378,1100000.0,None,NaN
1,tt0337692,On the Road,2012,124.0,"Adventure,Drama,Romance",6.1,37886,744000.0,8000000,8744000.0
2,tt0359950,The Secret Life of Walter Mitty,2013,114.0,"Adventure,Comedy,Drama",7.3,275300,58200000.0,129900000,188100000.0
3,tt0365907,A Walk Among the Tombstones,2014,114.0,"Action,Crime,Drama",6.5,105116,26300000.0,26900000,53200000.0
4,tt0369610,Jurassic World,2015,124.0,"Action,Adventure,Sci-Fi",7.0,539338,652300000.0,1019.4,652301019.4


In [32]:
df1.describe()

,start_year,runtime_minutes,averagerating,numvotes,domestic_gross,Total_revenues
count,3027.000000,2980.000000,3027.000000,3.027000e+03,3.005000e+03,1.810000e+03
mean,2013.783284,107.217114,6.457582,6.170030e+04,3.064033e+07,1.287922e+08
std,2.466955,20.073886,1.012277,1.255132e+05,6.671629e+07,2.085336e+08
min,2010.000000,3.000000,1.600000,5.000000e+00,1.000000e+02,1.080000e+04
25%,2012.000000,94.000000,5.900000,2.117000e+03,1.390000e+05,1.001450e+07
50%,2014.000000,105.000000,6.600000,1.310900e+04,2.000000e+06,4.813350e+07
75%,2016.000000,118.000000,7.100000,6.276550e+04,3.250000e+07,1.489000e+08
max,2019.000000,272.000000,9.200000,1.841066e+06,7.001000e+08,1.405400e+09


In [33]:
# Final Cleaning and Feature Engineering 
# 1. Convert total revenues to millions (M USD) for readability
df1['Total_revenues_mil'] = df1['Total_revenues'] / 1_000_000

In [34]:
#  Handle missing values in critical columns
# Drop rows where genres or total revenues are missing.
df1.dropna(subset=['genres', 'Total_revenues'], inplace=True)
df1['Total_revenues_mil'].isnull().sum()

0

In [35]:
# Impute missing runtime with the median for robust analysis
median_runtime = df1['runtime_minutes'].median()
df1['runtime_minutes'].fillna(median_runtime, inplace=True)

In [36]:
# Filter out films with 0 revenue 
df1 = df1[df1['Total_revenues_mil'] > 0]
print(f"Final Clean Rows for Analysis: {len(df1)}")
df1.info()

Final Clean Rows for Analysis: 1803
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1803 entries, 1 to 3026
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1803 non-null   object 
 1   primary_title       1803 non-null   object 
 2   start_year          1803 non-null   int64  
 3   runtime_minutes     1803 non-null   float64
 4   genres              1803 non-null   object 
 5   averagerating       1803 non-null   float64
 6   numvotes            1803 non-null   int64  
 7   domestic_gross      1803 non-null   float64
 8   foreign_gross       1803 non-null   object 
 9   Total_revenues      1803 non-null   float64
 10  Total_revenues_mil  1803 non-null   float64
dtypes: float64(5), int64(2), object(4)
memory usage: 169.0+ KB
